# Following steps are derived from Tensorflow [codelab](https://codelabs.developers.google.com/?cat=TensorFlow) [one](https://codelabs.developers.google.com/codelabs/tensorflow-for-poets/index.html#0) and [two](https://codelabs.developers.google.com/codelabs/tensorflow-for-poets-2-tflite/index.html#0)

This tutorial assumes you have installed tensorflow from [here](https://www.tensorflow.org/install/)

## 1. Clone the git repository

All the code used in this codelab is contained in this git repository. 
Clone the repository and cd into it. This is where we will be working.

In [ ]:
git clone https://github.com/googlecodelabs/tensorflow-for-poets-2

cd tensorflow-for-poets-2

## 2. Download [FIDS30](http://www.vicos.si/Downloads/FIDS30) dataset

Rename the `FIDS30` folder to `fruit_photos` and move it in `tf_files` directory

In [ ]:
ls tf_files/fruit_photos

## 3. Retrain your MobileNet
Details [here](https://codelabs.developers.google.com/codelabs/tensorflow-for-poets/index.html#3).<br>
Configure the mobileNet to be trained

In [ ]:
IMAGE_SIZE=224
ARCHITECTURE="mobilenet_1.0_${IMAGE_SIZE}"

Add the following lines to `create_image_lists()` function in `retrain.py` script

In [ ]:
# make sure none of the list is empty, otherwise it will raise an error
# when validating / testing
if validation_percentage > 0 and not validation_images:
  validation_images.append(training_images.pop())
if testing_percentage > 0 and not testing_images:
  testing_images.append(training_images.pop())

Run the `retrain.py` script with appropriate parameters

In [ ]:
python -m scripts.retrain \
--bottleneck_dir=tf_files/bottlenecks \
--how_many_training_steps=500 \
--model_dir=tf_files/models/ \
--summaries_dir=tf_files/training_summaries/"${ARCHITECTURE}" \
--output_graph=tf_files/retrained_graph.pb \
--output_labels=tf_files/retrained_labels.txt \
--architecture="${ARCHITECTURE}" \
--image_dir=tf_files/fruit_photos \
--flip_left_right \
--random_crop 50 \
--random_scale 60 \
--random_brightness 60

## 4. Evaluate your training
Specify your own paths

In [ ]:
python -m scripts.label_image --graph=tf_files/retrained_graph.pb --image=tf_files/IMG_20180108_153200.jpg

## 5. Install TensorFlow: tf-nightly
More details [here](https://www.tensorflow.org/install/install_linux)

In [ ]:
sudo apt-get install python3-pip python3-dev python-virtualenv # for Python 3.n
virtualenv --system-site-packages -p python3 tf-nightly # for Python 3.n
source ~/tf-nightly/bin/activate # bash, sh, ksh, or zsh
pip3 install --upgrade tf-nightly     # for Python 3.n without GPU

### Optimize your model

In [ ]:
IMAGE_SIZE=224
toco \
--input_file=tf_files/retrained_graph.pb \
--output_file=tf_files/optimized_graph.pb \
--input_format=TENSORFLOW_GRAPHDEF \
--output_format=TENSORFLOW_GRAPHDEF \
--input_shape=1,${IMAGE_SIZE},${IMAGE_SIZE},3 \
--input_array=input \
--output_array=final_result

### Verify optimized model

In [ ]:
python -m scripts.label_image \
--graph=tf_files/optimized_graph.pb \
--image=tf_files/IMG_20180108_153200.jpg

## 6. Optimize the model to tf-lite format

In [ ]:
IMAGE_SIZE=224
toco \
--input_file=tf_files/retrained_graph.pb \
--output_file=tf_files/optimized_graph.lite \
--input_format=TENSORFLOW_GRAPHDEF \
--output_format=TFLITE \
--input_shape=1,${IMAGE_SIZE},${IMAGE_SIZE},3 \
--input_array=input \
--output_array=final_result \
--inference_type=FLOAT \
--input_type=FLOAT

## 7. (Optional) Investigate the changes with TensorBoard

In [ ]:
python -m scripts.graph_pb2tb tf_files/training_summaries/retrained \
tf_files/retrained_graph.pb 

python -m scripts.graph_pb2tb tf_files/training_summaries/optimized \
tf_files/optimized_graph.pb 
    
pkill -f tensorboard
tensorboard --logdir tf_files/training_summaries &

## 8. Setup android app
More details [here](https://codelabs.developers.google.com/codelabs/tensorflow-for-poets-2-tflite/index.html#4).
In the file selector, choose `tensorflow-for-poets-2/android/tflite` from your working directory.

### Add your model files to the project

In [ ]:
cp tf_files/optimized_graph.lite android/tflite/app/src/main/assets/graph.lite 
cp tf_files/retrained_labels.txt android/tflite/app/src/main/assets/labels.txt 

# 9. DONE !